In [1]:
import scipy.io
import numpy as np
import CMR_IA as cmr
import matplotlib.pyplot as plt
import pickle
from statistics import mean, stdev
import pprint
# import import_ipynb
import time
import pandas as pd
import seaborn as sns

np.set_printoptions(edgeitems=20, linewidth=150,precision=4)

In [38]:
with open("../Data/simu6c_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)

In [39]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [40]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0.5,
    beta_rec_post = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.01,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0.01,
    eta = 0,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 10,
    d_ass = 1,
    learn_while_retrieving = True
)
params

{'beta_enc': 0.5,
 'beta_rec': 0.5,
 'beta_rec_new': 0.5,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0,
 'omega': 8,
 'alpha': 4,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.01,
 'd_ass': 1,
 'lamb': 0.01,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 10,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [41]:
df_study = df_study.loc[df_study.session == 0]
# df_study['study_itemno1'] = [1,3,5,7,9,11,13,15,17,19,21,23]
# df_study['study_itemno2'] = [2,4,6,8,10,12,14,16,18,20,22,24]
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session
0,140,838,BLUEBERRY,LOCK,0
1,1587,1240,WARDROBE,SAXOPHONE,0
2,1132,702,PROTON,HELMET,0
3,299,1282,CHILD,SHELTER,0
4,1391,1059,STRAWBERRY,PICKLE,0
5,153,736,BOTTLE,INDIAN,0
6,145,1198,BODY,ROBE,0
7,819,1251,LIGHTER,SCOTCH,0
8,711,1519,HOE,TUNNEL,0
9,607,856,FRIES,MAID,0


In [42]:
df_test = df_test.loc[df_test.session == 0]
# df_test['test_itemno'] = [1]
# df_test['correct_ans'] = [2]
df_test

,test_itemno,test_item,correct_ans,order,rep,test,session,test_pos,pair_idx
0,210,CABBAGE,327,1,1,1,0,1,11
1,1251,SCOTCH,819,2,1,1,0,2,8
2,145,BODY,1198,1,1,1,0,3,7
3,1059,PICKLE,1391,2,1,1,0,4,5
4,711,HOE,1519,1,1,1,0,5,9
5,838,LOCK,140,2,1,1,0,6,1
6,153,BOTTLE,736,1,1,1,0,7,6
7,1587,WARDROBE,1240,1,1,1,0,8,2
8,856,MAID,607,2,1,1,0,9,10
9,702,HELMET,1132,2,1,1,0,10,3


In [43]:
df_thin = df_test[['session','test_itemno']]
df_thin = df_thin.assign(s_resp=np.nan, s_rt=np.nan)
df_thin

,session,test_itemno,s_resp,s_rt
0,0,210,NaN,NaN
1,0,1251,NaN,NaN
2,0,145,NaN,NaN
3,0,1059,NaN,NaN
4,0,711,NaN,NaN
5,0,838,NaN,NaN
6,0,153,NaN,NaN
7,0,1587,NaN,NaN
8,0,856,NaN,NaN
9,0,702,NaN,NaN


In [44]:
# extarct the session data
sess = 0
pres_mat = df_study.loc[df_study.session==sess,['study_itemno1','study_itemno2']].to_numpy()
pres_mat = np.reshape(pres_mat,(1,len(pres_mat),2))
pres_mat

array([[[ 140,  838],
        [1587, 1240],
        [1132,  702],
        [ 299, 1282],
        [1391, 1059],
        [ 153,  736],
        [ 145, 1198],
        [ 819, 1251],
        [ 711, 1519],
        [ 607,  856],
        [ 210,  327],
        [1169, 1372]]], dtype=int64)

In [45]:
cue_mat = df_thin.loc[df_thin.session==sess,'test_itemno'].to_numpy()
cue_mat

array([ 210, 1251,  145, 1059,  711,  838,  153, 1587,  856,  702,  299, 1372,  140,  145, 1169,  299, 1059, 1519,  210, 1132, 1240, 1251,  856,
        736], dtype=int64)

In [46]:
sem_mat = s_mat

In [47]:
model = cmr.CMR2(params, pres_mat, sem_mat, source_mat=None,
           rec_mat=None, ffr_mat=None, cue_mat=cue_mat, task='CR', mode='Final')

In [52]:
model.pres_nos

array([[[ 140,  838],
        [1587, 1240],
        [1132,  702],
        [ 299, 1282],
        [1391, 1059],
        [ 153,  736],
        [ 145, 1198],
        [ 819, 1251],
        [ 711, 1519],
        [ 607,  856],
        [ 210,  327],
        [1169, 1372]]], dtype=int16)

In [53]:
model.cues_nos

array([ 210, 1251,  145, 1059,  711,  838,  153, 1587,  856,  702,  299, 1372,  140,  145, 1169,  299, 1059, 1519,  210, 1132, 1240, 1251,  856,
        736], dtype=int16)

In [54]:
model.all_nos_unique

array([ 140,  145,  153,  210,  299,  327,  607,  702,  711,  736,  819,  838,  856, 1059, 1132, 1169, 1198, 1240, 1251, 1282, 1372, 1391, 1519,
       1587], dtype=int16)

In [55]:
model.pres_indexes

array([[[ 0, 11],
        [23, 17],
        [14,  7],
        [ 4, 19],
        [21, 13],
        [ 2,  9],
        [ 1, 16],
        [10, 18],
        [ 8, 22],
        [ 6, 12],
        [ 3,  5],
        [15, 20]]], dtype=int64)

In [56]:
model.cues_indexes

array([ 3, 18,  1, 13,  8, 11,  2, 23, 12,  7,  4, 20,  0,  1, 15,  4, 13, 22,  3, 14, 17, 18, 12,  9], dtype=int64)

In [57]:
model.nlists

1

In [49]:
model.run_norm_cr_single_sess()

# in model:
# print(self.att_vec[self.all_nos_unique[item_idx]-1])
# print(np.dot(self.c_old[:self.nitems_unique], self.f[:self.nitems_unique].T))

In [50]:
model.ret_thresh

array([1.3422e+08, 3.3554e+07, 1.0000e+00, 3.2769e+04, 2.0972e+06, 2.0972e+06, 1.0000e+00, 1.0000e+00, 2.0490e+03, 3.3000e+01, 1.0000e+00,
       1.0000e+00, 1.2900e+02, 5.2429e+05, 8.1930e+03, 8.3886e+06, 8.3886e+06, 2.0490e+03, 5.1300e+02, 1.0000e+00, 1.0000e+00, 1.3107e+05,
       1.3107e+05, 9.0000e+00], dtype=float32)

In [58]:
model.rec_items

[327,
 819,
 1198,
 1391,
 1519,
 -1,
 -1,
 1240,
 -1,
 -1,
 -1,
 -1,
 1240,
 1198,
 327,
 210,
 1391,
 711,
 711,
 711,
 1587,
 1587,
 1587,
 1587]

In [59]:
model.rec_times

[1210,
 1520,
 1420,
 1530,
 1590,
 -1,
 -1,
 1610,
 -1,
 -1,
 -1,
 -1,
 1000,
 750,
 990,
 810,
 730,
 730,
 4490,
 3230,
 790,
 4450,
 3180,
 2580]

In [60]:
model.f_in

array([1.2319, 1.3632, 0.7212, 1.6321, 0.8016, 1.1183, 0.5213, 0.4077, 2.2097, 0.7016, 1.0256, 0.3108, 1.2987, 1.5312, 0.8191, 0.7453, 1.351 ,
       2.7616, 1.7708, 0.4245, 0.3696, 1.5284, 1.3671, 3.7656], dtype=float32)

In [61]:
model.M_FC

array([[ 7.1419e-01,  2.8536e-01,  1.2410e-01,  2.4821e-01,  2.1202e-01,  1.7180e-01,  7.7301e-02,  1.5919e-01,  3.3399e-01,  2.2769e-01,
         1.4076e-01,  4.9955e-01,  1.7121e-01,  3.0412e-01,  2.2506e-01,  1.3408e-01,  2.8383e-01,  1.2121e+00,  2.6876e-01,  1.3433e-01,
         5.5622e-02,  3.2252e-01,  2.0202e-01,  8.3440e-01,  0.0000e+00,  0.0000e+00],
       [ 3.2083e-02,  1.0405e+00,  1.9876e-02,  3.6197e-01,  1.1931e-01,  3.4960e-01,  1.3376e-01,  1.6927e-02,  5.0133e-01,  4.6452e-02,
         3.0387e-01,  2.2902e-03,  1.8724e-01,  1.7830e-01,  8.2914e-02,  2.5126e-01,  2.0506e+00,  1.0199e-01,  3.4533e-01,  1.3650e-02,
         9.6580e-02,  1.8219e-01,  3.5277e-01,  2.0736e-01,  0.0000e+00,  0.0000e+00],
       [ 5.6503e-02,  4.1892e-01,  5.2231e-01,  3.2093e-01,  8.9553e-02,  2.7243e-01,  1.3500e-01,  2.3410e-02,  4.4255e-01,  6.4823e-01,
         2.7911e-01,  5.2428e-03,  1.9184e-01,  1.3125e-01,  7.6470e-02,  1.7887e-01,  4.2212e-01,  1.8659e-01,  3.5013e-01,  2.0062e-02

In [23]:
model.M_CF

array([[0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1531, 0.1531, 0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1531, 0.1531, 0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1326, 0.1326, 0.1531, 0.1531, 0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1326, 0.1326, 0.1531, 0.1531, 0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1148, 0.1148, 0.1326, 

In [51]:
recs = model.rec_items
rts = model.rec_times
result = np.column_stack((recs,rts))
df_thin.loc[df_thin.session==sess, ['s_resp','s_rt']] = result
df_simu = df_thin.merge(df_test,on=['session','test_itemno'])
df_simu

,session,test_itemno,s_resp,s_rt,test_item,correct_ans,order,rep,test,test_pos,pair_idx
0,0,210,327.0,1210.0,CABBAGE,327,1,1,1,1,11
1,0,210,327.0,1210.0,CABBAGE,327,1,1,2,19,11
2,0,210,711.0,4490.0,CABBAGE,327,1,1,1,1,11
3,0,210,711.0,4490.0,CABBAGE,327,1,1,2,19,11
4,0,1251,819.0,1520.0,SCOTCH,819,2,1,1,2,8
5,0,1251,819.0,1520.0,SCOTCH,819,2,1,2,22,8
6,0,1251,1587.0,4450.0,SCOTCH,819,2,1,1,2,8
7,0,1251,1587.0,4450.0,SCOTCH,819,2,1,2,22,8
8,0,145,1198.0,1420.0,BODY,1198,1,1,1,3,7
9,0,145,1198.0,1420.0,BODY,1198,1,1,2,14,7


In [25]:
tmp1 = df_study.study_itemno1.to_numpy()
tmp2 = df_study.study_itemno2.to_numpy()
tmp = np.concatenate((tmp1, tmp2))
tmp = np.sort(tmp)
tmp

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [26]:
testid = np.searchsorted(tmp, df_simu.test_itemno.item())
corrid = np.searchsorted(tmp, df_simu.correct_ans.item())

In [27]:
df_simu['corr_fin']= model.f_in[corrid]
df_simu['othe_fin'] = np.mean(np.delete(model.f_in, [testid,corrid]))
df_simu

,session,test_itemno,s_resp,s_rt,test_item,correct_ans,lag,corr_fin,othe_fin
0,0,1,2.0,1960.0,HOOP,2,5,0.63972,0.351864
